## Near-infrared spectroscopic assessment of malaria parasitemia in blood

In [ ]:
# Load the drive helper and mount
from google.colab import drive

# This will prompt for authorization
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the drive helper and mount
from google.colab import drive

# This will prompt for authorization
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd ./drive/My Drive/data/blood
!ls

[Errno 2] No such file or directory: './drive/My Drive/data/blood'
/content
drive  sample_data


In [ ]:

# Standard data manipulation and plotting
import numpy as np
import seaborn as sns; sns.set(color_codes=True)
from pandas import read_csv, DataFrame, concat
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
# Machine and deep learning
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
import keras.backend as K

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
# Classification libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import itertools
from sklearn.metrics import plot_confusion_matrix
import pickle
# Computer vision and others
# import cv2
import glob
import os
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')
# "error", "ignore", "always", "default", "module" or "once"
print(__doc__)

#### Analysis of whole blood

In [ ]:
#Load dataset 
meta = read_csv('blood/wbc_meta.csv', sep=',')
rbcClasses = meta['Class']
rbcTarget = meta['Parasitemia (%)']
rbcNames = meta['Name']
del(meta)

spectra = read_csv('blood/wbc_nir.csv', sep=',')
# spectra = read_csv('data/wbc_whole.csv', sep=',')
spectra.head()#Load dataset 
meta = read_csv('blood/wbc_meta.csv', sep=',')
rbcClasses = meta['Class']
rbcTarget = meta['Parasitemia (%)']
rbcNames = meta['Name']
del(meta)

spectra = read_csv('blood/wbc_nir.csv', sep=',')
# spectra = read_csv('data/wbc_whole.csv', sep=',')
spectra.head()

FileNotFoundError: ignored

In [ ]:
print(len(spectra))

### Nested cross-validation

Use spectral data and parasitemia from one biological sample per class in nested cross-validation to test spectra - parasitemia relationship.

In [ ]:
uniqNames = rbcNames.unique()
indexn = []

for i in range(len(uniqNames)):

  if uniqNames[i][0:2] == 'R3':
    indexn.append(uniqNames[i])

print('R1 replicates are: ', indexn)
sampletestInd = []
for i in range(len(indexn)):
  sampleTest = rbcNames[rbcNames == indexn[i]].index.values.tolist()
  sampletestInd.append(sampleTest)
  R3_nested_test_ind = list(itertools.chain.from_iterable(sampletestInd))

# print(len(R1_nested_test_ind))
# print(len(R2_nested_test_ind))
print(len(R3_nested_test_ind))


In [ ]:
print('Number of replicates in R1, R2 and R3 are:', len(R1_nested_test_ind), ',', len(R2_nested_test_ind), ' and ', len(R3_nested_test_ind), 'respectively.')
print('Total sample count = ', len(R1_nested_test_ind)+len(R2_nested_test_ind)+len(R3_nested_test_ind))

In [ ]:

tot_list = rbcNames.index.values.tolist()

R1_nested_train_ind = [x for x in tot_list if x not in R1_nested_test_ind]
print('Number of training and test samples in with R1 nested: ', len(R1_nested_train_ind), 'and', len(R1_nested_test_ind))

R2_nested_train_ind = [x for x in tot_list if x not in R2_nested_test_ind]
print('Number of training and test samples in with R2 nested: ', len(R2_nested_train_ind), 'and', len(R2_nested_test_ind))

R3_nested_train_ind = [x for x in tot_list if x not in R3_nested_test_ind]
print('Number of training and test samples in with R3 nested: ', len(R3_nested_train_ind), 'and', len(R3_nested_test_ind))


In [ ]:
R1_nest = [R1_nested_train_ind, R1_nested_test_ind]
R2_nest = [R2_nested_train_ind, R2_nested_test_ind]
R3_nest = [R3_nested_train_ind, R3_nested_test_ind]

nests = [R1_nest, R2_nest, R3_nest]

for nest in nests:
  print('Number of training sample in this nest is:', len(nest[0]))
  print('Number of test samples in this nest is:', len(nest[1]))

##### 1. Support Vector Classifier (SVC)

In [ ]:
'''
Loop through all nests 
'''
X = spectra.values
le = LabelEncoder()
y_class = le.fit_transform(rbcClasses)
print('List of classes:', list(le.classes_))

nested_models = []
nest_count = 0

# Create the parameter grid 
param_grid = {
    'C':[0.1,1,10,100,1000], #
    'gamma':[0.0001,0.001,0.01,0.1,1,10],# 
    'kernel':['linear','rbf','poly','sigmoid'],
    'degree':[1,2,4]}


for nest in nests:
  X_train = X[nest[0]]
  X_test = X[nest[1]]
  y_train = y_class[nest[0]]
  y_test = y_class[nest[1]]

  # initialize classifier algorithm
  svc = SVC(random_state = 42)

  # create model gridsearch
  model_svc = GridSearchCV(estimator = svc, scoring='accuracy', param_grid = param_grid,
                     cv = 5, n_jobs = -1, verbose = 2, return_train_score=True)
  
  # Train model on training dataset
  print('Running nest: ', "nest_" + str(nest_count))
  model_svc.fit(X_train, y_train)

  best_model = model_svc.best_estimator_
  y_pred_train = best_model.predict(X_train)
  y_pred_test = best_model.predict(X_test)

  # Append each nest and corresponding model to dictionary
  nested_models.append({"nest" : [model_svc, X_train, y_train, y_pred_train, X_test, y_test, y_pred_test]})
  # nested_models.append({"nest_" + str(nest_count): [model_svc, X_train, y_train, y_pred_train, X_test, y_test, y_pred_test]})
  nest_count += 1
  del(model_svc)




In [ ]:
# CM for training set
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
nest = nested_models[2]
spec_nest = 'nest'

model = nest[spec_nest][0]
X_train = nest[spec_nest][1]
y_train_nest_1 = nest[spec_nest][2]
y_pred_train_nest_1 = nest[spec_nest][3]
X_test = nest[spec_nest][4]
y_test_nest_1 = nest[spec_nest][5]
y_pred_test_nest_1 = nest[spec_nest][6]

print("Classification report for training")
print(classification_report(y_train_nest_1, y_pred_train_nest_1, target_names=class_names))

cm = confusion_matrix(y_train_nest_1, y_pred_train_nest_1)
np.set_printoptions(suppress=True)

conf = plot_confusion_matrix(model.best_estimator_, X_train, y_train_nest_1,
                             display_labels=class_names,
                             cmap=plt.cm.Blues,values_format='.1g',
                             normalize='true')
#print(conf.confusion_matrix)
plt.grid(None)
plt.show()
# plt.savefig('wbc_whole_svc_train.png', dpi=900)
print( ' ')
print( '===================== ')
print( ' ')
print("Classification report for test")
print(classification_report(y_test_nest_1, y_pred_test_nest_1, target_names=class_names))

cm = confusion_matrix(y_test_nest_1, y_pred_test_nest_1)
np.set_printoptions(suppress=True)

conf = plot_confusion_matrix(model.best_estimator_, X_test, y_test_nest_1,
                             display_labels=class_names,
                             cmap=plt.cm.Blues,values_format='.1g',
                             normalize='true')
#print(conf.confusion_matrix)
plt.grid(None)
plt.show()

##### 2. Logistic Regression (LR)

In [ ]:
'''
Loop through all nests 
'''
X = spectra.values
le = LabelEncoder()
y_class = le.fit_transform(rbcClasses)
print('List of classes:', list(le.classes_))

nested_lr_models = []
nest_count = 0

# Create the parameter grid 
param_grid = {
    'penalty': ['l1','l2','elasticnet'], 
    'C': [0.001,0.01,0.1,1,10,100,1000],
    'solver': ['newton-cg','lbfgs','liblinear','sag','saga']
    }


for nest in nests:
  X_train = X[nest[0]]
  X_test = X[nest[1]]
  y_train = y_class[nest[0]]
  y_test = y_class[nest[1]]

  # create model gridsearch  
  model_lr = GridSearchCV(LR(random_state=0), scoring='accuracy', param_grid = param_grid, 
                       n_jobs=-1, cv=5, verbose=2)
  
  # Train model on training dataset
  print('Running nest: ', "nest_" + str(nest_count))
  model_lr.fit(X_train, y_train)

  best_model = model_lr.best_estimator_
  y_pred_train = best_model.predict(X_train)
  y_pred_test = best_model.predict(X_test)

  # Append each nest and corresponding model to dictionary
  nested_lr_models.append({"nest" : [model_lr, X_train, y_train, y_pred_train, X_test, y_test, y_pred_test]})
  # nested_models.append({"nest_" + str(nest_count): [model_svc, X_train, y_train, y_pred_train, X_test, y_test, y_pred_test]})
  nest_count += 1
  del(model_lr)

In [ ]:
# CM for training set
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
nest = nested_lr_models[2]
spec_nest = 'nest'

model = nest[spec_nest][0]
X_train = nest[spec_nest][1]
y_train_nest_1 = nest[spec_nest][2]
y_pred_train_nest_1 = nest[spec_nest][3]
X_test = nest[spec_nest][4]
y_test_nest_1 = nest[spec_nest][5]
y_pred_test_nest_1 = nest[spec_nest][6]

print("Classification report for training")
print(classification_report(y_train_nest_1, y_pred_train_nest_1, target_names=class_names))

cm = confusion_matrix(y_train_nest_1, y_pred_train_nest_1)
np.set_printoptions(suppress=True)

conf = plot_confusion_matrix(model.best_estimator_, X_train, y_train_nest_1,
                             display_labels=class_names,
                             cmap=plt.cm.Blues,values_format='.1g',
                             normalize='true')
#print(conf.confusion_matrix)
plt.grid(None)
plt.show()
# plt.savefig('wbc_whole_svc_train.png', dpi=900)
print( ' ')
print( '===================== ')
print( ' ')
print("Classification report for test")
print(classification_report(y_test_nest_1, y_pred_test_nest_1, target_names=class_names))

cm = confusion_matrix(y_test_nest_1, y_pred_test_nest_1)
np.set_printoptions(suppress=True)

conf = plot_confusion_matrix(model.best_estimator_, X_test, y_test_nest_1,
                             display_labels=class_names,
                             cmap=plt.cm.Blues,values_format='.1g',
                             normalize='true')
#print(conf.confusion_matrix)
plt.grid(None)
plt.show()

#### Comparison of SVM and LR for all nested combinations

In [ ]:
acc_svm_full = [41, 97, 94]
acc_lr_full = [65, 100, 100]

acc_svm_uv = [74, 100, 100]
acc_lr_uv = [68, 100, 94]

acc_svm_nir = [26, 97, 100]
acc_lr_nir = [56, 100, 94]

In [ ]:

n_rows = 3
columns = ('Nest_1','Nest_2','Nest_3')

ind = np.arange(n_rows) 
width = 0.35 

# UV
fig = plt.figure(figsize=(18,5))
plt.subplot(131)
plt.bar(ind, acc_svm_uv, width, color = 'b',label='SVM')
plt.bar(ind + width, acc_lr_uv, width, color = 'g', label='LR')

plt.ylabel('Accuracy')
plt.title('Accuracy of WBC nested models - UV range')

plt.xticks(ind + width / 2, columns)
plt.legend(loc='best')

# NIR
plt.subplot(132)
plt.bar(ind, acc_svm_nir, width, color = 'b',label='SVM')
plt.bar(ind + width, acc_lr_nir, width, color = 'g', label='LR')

plt.ylabel('Accuracy')
plt.title('Accuracy of WBC nested models - NIR range')

plt.xticks(ind + width / 2, columns)
# plt.legend(loc='best')


# FULL SPECTRUM
plt.subplot(133)
plt.bar(ind, acc_svm_full, width, color = 'b',label='SVM')
plt.bar(ind + width, acc_lr_full, width, color = 'g', label='LR')

plt.ylabel('Accuracy')
plt.title('Accuracy of WBC nested models - full range')

plt.xticks(ind + width / 2, columns)
# plt.legend(loc='best')
# plt.show()

plt.savefig('acc_wbc.png', dpi=900)

#### Analysis of red blood cells

In [ ]:
#Load dataset 
meta = read_csv('data/rbc_meta.csv', sep=',')
rbcClasses = meta['Class']
rbcTarget = meta['Parasitemia (%)']
rbcNames = meta['Name']
del(meta)

spectra = read_csv('data/rbc_nir.csv', sep=',')
spectra.head()

In [ ]:
print(len(spectra))

### Nested cross-validation

Use spectral data and parasitemia from one biological sample per class in nested cross-validation to test spectra - parasitemia relationship.

In [ ]:
uniqNames = rbcNames.unique()

indexn = []

for i in range(len(uniqNames)):

  if uniqNames[i][0:2] == 'R1':
    indexn.append(uniqNames[i])

print('R1 replicates are: ', indexn)
sampletestInd = []
for i in range(len(indexn)):
  sampleTest = rbcNames[rbcNames == indexn[i]].index.values.tolist()
  sampletestInd.append(sampleTest)
  R1_nested_test_ind = list(itertools.chain.from_iterable(sampletestInd))

print(len(R1_nested_test_ind))
# print(len(R2_nested_test_ind))
# print(len(R3_nested_test_ind))


In [ ]:
print('Number of replicates in R1, R2 and R3 are:', len(R1_nested_test_ind), ',', len(R2_nested_test_ind), ' and ', len(R3_nested_test_ind), 'respectively.')
print('Total sample count = ', len(R1_nested_test_ind)+len(R2_nested_test_ind)+len(R3_nested_test_ind))

In [ ]:

tot_list = rbcNames.index.values.tolist()

R1_nested_train_ind = [x for x in tot_list if x not in R1_nested_test_ind]
print('Number of training and test samples in with R1 nested: ', len(R1_nested_train_ind), 'and', len(R1_nested_test_ind))

R2_nested_train_ind = [x for x in tot_list if x not in R2_nested_test_ind]
print('Number of training and test samples in with R2 nested: ', len(R2_nested_train_ind), 'and', len(R2_nested_test_ind))

R3_nested_train_ind = [x for x in tot_list if x not in R3_nested_test_ind]
print('Number of training and test samples in with R3 nested: ', len(R3_nested_train_ind), 'and', len(R3_nested_test_ind))


In [ ]:
R1_nest = [R1_nested_train_ind, R1_nested_test_ind]
R2_nest = [R2_nested_train_ind, R2_nested_test_ind]
R3_nest = [R3_nested_train_ind, R3_nested_test_ind]

nests = [R1_nest, R2_nest, R3_nest]

for nest in nests:
  print('Number of training sample in this nest is:', len(nest[0]))
  print('Number of test samples in this nest is:', len(nest[1]))

##### 1. Support Vector Classifier (SVC)

In [ ]:
'''
Loop through all nests 
'''
X = spectra.values
le = LabelEncoder()
y_class = le.fit_transform(rbcClasses)
print('List of classes:', list(le.classes_))

nested_models = []
nest_count = 0

# Create the parameter grid 
param_grid = {
    'C':[0.1,1,10,100,1000], #
    'gamma':[0.0001,0.001,0.01,0.1,1,10],# 
    'kernel':['linear','rbf','poly','sigmoid'],
    'degree':[1,2,4]}


for nest in nests:
  X_train = X[nest[0]]
  X_test = X[nest[1]]
  y_train = y_class[nest[0]]
  y_test = y_class[nest[1]]

  # initialize classifier algorithm
  svc = SVC(random_state = 42)

  # create model gridsearch
  model_svc = GridSearchCV(estimator = svc, scoring='accuracy', param_grid = param_grid,
                     cv = 5, n_jobs = -1, verbose = 2, return_train_score=True)
  
  # Train model on training dataset
  print('Running nest: ', "nest_" + str(nest_count))
  model_svc.fit(X_train, y_train)

  best_model = model_svc.best_estimator_
  y_pred_train = best_model.predict(X_train)
  y_pred_test = best_model.predict(X_test)

  # Append each nest and corresponding model to dictionary
  nested_models.append({"nest" : [model_svc, X_train, y_train, y_pred_train, X_test, y_test, y_pred_test]})
  # nested_models.append({"nest_" + str(nest_count): [model_svc, X_train, y_train, y_pred_train, X_test, y_test, y_pred_test]})
  nest_count += 1
  del(model_svc)

In [ ]:
# CM for training set
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
nest = nested_models[2]
spec_nest = 'nest'

model = nest[spec_nest][0]
X_train = nest[spec_nest][1]
y_train_nest_1 = nest[spec_nest][2]
y_pred_train_nest_1 = nest[spec_nest][3]
X_test = nest[spec_nest][4]
y_test_nest_1 = nest[spec_nest][5]
y_pred_test_nest_1 = nest[spec_nest][6]

print("Classification report for training")
print(classification_report(y_train_nest_1, y_pred_train_nest_1, target_names=class_names))

cm = confusion_matrix(y_train_nest_1, y_pred_train_nest_1)
np.set_printoptions(suppress=True)

conf = plot_confusion_matrix(model.best_estimator_, X_train, y_train_nest_1,
                             display_labels=class_names,
                             cmap=plt.cm.Blues,values_format='.1g',
                             normalize='true')
#print(conf.confusion_matrix)
plt.grid(None)
plt.show()
# plt.savefig('wbc_whole_svc_train.png', dpi=900)
print( ' ')
print( '===================== ')
print( ' ')
print("Classification report for test")
print(classification_report(y_test_nest_1, y_pred_test_nest_1, target_names=class_names))

cm = confusion_matrix(y_test_nest_1, y_pred_test_nest_1)
np.set_printoptions(suppress=True)

conf = plot_confusion_matrix(model.best_estimator_, X_test, y_test_nest_1,
                             display_labels=class_names,
                             cmap=plt.cm.Blues,values_format='.1g',
                             normalize='true')
#print(conf.confusion_matrix)
plt.grid(None)
plt.show()



##### 2. Logistic Regression (LR)

In [ ]:
'''
Loop through all nests 
'''
X = spectra.values
le = LabelEncoder()
y_class = le.fit_transform(rbcClasses)
print('List of classes:', list(le.classes_))

nested_lr_models = []
nest_count = 0

# Create the parameter grid 
param_grid = {
    'penalty': ['l1','l2','elasticnet'], 
    'C': [0.001,0.01,0.1,1,10,100,1000],
    'solver': ['newton-cg','lbfgs','liblinear','sag','saga']
    }


for nest in nests:
  X_train = X[nest[0]]
  X_test = X[nest[1]]
  y_train = y_class[nest[0]]
  y_test = y_class[nest[1]]

  # create model gridsearch  
  model_lr = GridSearchCV(LR(random_state=0), scoring='accuracy', param_grid = param_grid, 
                       n_jobs=-1, cv=5, verbose=2)
  
  # Train model on training dataset
  print('Running nest: ', "nest_" + str(nest_count))
  model_lr.fit(X_train, y_train)

  best_model = model_lr.best_estimator_
  y_pred_train = best_model.predict(X_train)
  y_pred_test = best_model.predict(X_test)

  # Append each nest and corresponding model to dictionary
  nested_lr_models.append({"nest" : [model_lr, X_train, y_train, y_pred_train, X_test, y_test, y_pred_test]})
  # nested_models.append({"nest_" + str(nest_count): [model_svc, X_train, y_train, y_pred_train, X_test, y_test, y_pred_test]})
  nest_count += 1
  del(model_lr)

In [ ]:
# CM for training set
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
nest = nested_lr_models[2]
spec_nest = 'nest'

model = nest[spec_nest][0]
X_train = nest[spec_nest][1]
y_train_nest_1 = nest[spec_nest][2]
y_pred_train_nest_1 = nest[spec_nest][3]
X_test = nest[spec_nest][4]
y_test_nest_1 = nest[spec_nest][5]
y_pred_test_nest_1 = nest[spec_nest][6]

print("Classification report for training")
print(classification_report(y_train_nest_1, y_pred_train_nest_1, target_names=class_names))

cm = confusion_matrix(y_train_nest_1, y_pred_train_nest_1)
np.set_printoptions(suppress=True)

conf = plot_confusion_matrix(model.best_estimator_, X_train, y_train_nest_1,
                             display_labels=class_names,
                             cmap=plt.cm.Blues,values_format='.1g',
                             normalize='true')
#print(conf.confusion_matrix)
plt.grid(None)
plt.show()
# plt.savefig('wbc_whole_svc_train.png', dpi=900)
print( ' ')
print( '===================== ')
print( ' ')
print("Classification report for test")
print(classification_report(y_test_nest_1, y_pred_test_nest_1, target_names=class_names))

cm = confusion_matrix(y_test_nest_1, y_pred_test_nest_1)
np.set_printoptions(suppress=True)

conf = plot_confusion_matrix(model.best_estimator_, X_test, y_test_nest_1,
                             display_labels=class_names,
                             cmap=plt.cm.Blues,values_format='.1g',
                             normalize='true')
#print(conf.confusion_matrix)
plt.grid(None)
plt.show()


#### Comparison of SVM and LR for all nested combinations

In [ ]:
acc_svm_rbc_full = [67, 100, 92]
acc_lr_rbc_full = [71, 100, 86]

acc_svm_rbc_uv = [67, 100, 91]
acc_lr_rbc_uv = `[64, 100, 84]

acc_svm_rbc_nir = [51, 100, 89]
acc_lr_rbc_nir = [64, 100, 88]

In [ ]:

n_rows = 3
columns = ('Nest_1','Nest_2','Nest_3')

ind = np.arange(n_rows) 
width = 0.35 

# UV
fig = plt.figure(figsize=(18,5))
plt.subplot(131)
plt.bar(ind, acc_svm_rbc_uv, width, color = 'b',label='SVM')
plt.bar(ind + width, acc_lr_rbc_uv, width, color = 'g', label='LR')

plt.ylabel('Accuracy')
plt.title('Accuracy of RBC nested models - UV range')

plt.xticks(ind + width / 2, columns)
plt.legend(loc='best')

# NIR
plt.subplot(132)
plt.bar(ind, acc_svm_rbc_nir, width, color = 'b',label='SVM')
plt.bar(ind + width, acc_lr_rbc_nir, width, color = 'g', label='LR')

plt.ylabel('Accuracy')
plt.title('Accuracy of RBC nested models - NIR range')

plt.xticks(ind + width / 2, columns)
# plt.legend(loc='best')


# FULL SPECTRUM
plt.subplot(133)
plt.bar(ind, acc_svm_rbc_full, width, color = 'b',label='SVM')
plt.bar(ind + width, acc_lr_rbc_full, width, color = 'g', label='LR')

plt.ylabel('Accuracy')
plt.title('Accuracy of RBC nested models - full range')

plt.xticks(ind + width / 2, columns)
# plt.legend(loc='best')
# plt.show()

plt.savefig('acc_rbc.png', dpi=900)